In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers import initializers
from tensorflow.examples.tutorials.mnist import input_data
from math import sqrt

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.0001
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = './tensorflow_logs/mnist'
# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 28*28 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [4]:
x = tf.placeholder(tf.float32, shape=[None, n_input], name='InputData')
y = tf.placeholder(tf.float32, shape=[None, n_classes], name='LabelData')
x_image = tf.reshape(x, [-1,28,28,1], name='image')

In [5]:
def NN(inputs):
    with slim.arg_scope([slim.fully_connected, slim.conv2d], 
                        activation_fn=tf.nn.relu,
                        weights_initializer=initializers.xavier_initializer()
                       ):
        out = slim.conv2d(inputs, 32, [3, 3])
        print (out.get_shape())
        out = slim.max_pool2d(out, [2, 2], [2, 2])
        print (out.get_shape())
        out = slim.conv2d(inputs, 64, [3, 3])
        print (out.get_shape())
        out = slim.max_pool2d(out, [2, 2], [2, 2])
        print (out.get_shape())
        out = slim.flatten(out)
        print (out.get_shape())
        out = slim.fully_connected(out, 256)
        print (out.get_shape())
        out = slim.fully_connected(out, 256)
        print (out.get_shape())
        out = slim.fully_connected(out, 10, activation_fn=None, normalizer_fn=None)
    return out

In [6]:
with tf.name_scope('Model'):
    # Model
    pred = NN(x_image)
with tf.name_scope('Cost'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

(?, 28, 28, 32)
(?, 14, 14, 32)
(?, 28, 28, 64)
(?, 14, 14, 64)
(?, 12544)
(?, 256)
(?, 256)


In [7]:
mnist.test.images.shape

(10000, 784)

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.scalar_summary("cost", cost)
# Create a summary to monitor accuracy tensor
tf.scalar_summary("accuracy", accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.merge_all_summaries()

In [ ]:
%%time
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, acc, summary = sess.run([optimizer, cost, accuracy, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost =", "{:.9f}".format(avg_cost), "acc =", "{:.9f}".format(acc))

    print ("Optimization Finished!")

    # Test model
    # Calculate accuracy
    test_xs, test_ys = mnist.test.next_batch(3000)
    print ('Final accuracy : %s' % sess.run(accuracy, feed_dict={x: test_xs ,y: test_ys}))

    print ("Run the command line:\n" \
          "--> tensorboard --logdir=./tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Epoch: 0001 cost = 0.437447676 acc = 0.960000038
Epoch: 0002 cost = 0.154766474 acc = 0.979999959
Epoch: 0003 cost = 0.101445435 acc = 0.960000038
Epoch: 0004 cost = 0.074161657 acc = 0.970000029
Epoch: 0005 cost = 0.058629182 acc = 0.989999950
Epoch: 0006 cost = 0.048001077 acc = 0.979999959
Epoch: 0007 cost = 0.040303289 acc = 1.000000000
Epoch: 0008 cost = 0.034975828 acc = 0.970000029
Epoch: 0009 cost = 0.029890664 acc = 0.989999950
Epoch: 0010 cost = 0.025463971 acc = 0.989999950
Epoch: 0011 cost = 0.021628636 acc = 0.989999950
Epoch: 0012 cost = 0.018641995 acc = 1.000000000
Epoch: 0013 cost = 0.016536066 acc = 0.999999940
Epoch: 0014 cost = 0.013843616 acc = 0.980000019
Epoch: 0015 cost = 0.011988339 acc = 0.990000010
Epoch: 0016 cost = 0.011519553 acc = 1.000000000
Epoch: 0017 cost = 0.009677692 acc = 1.000000000
Epoch: 0018 cost = 0.006960263 acc = 1.000000000
